# Week H

More Neural Networks

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py

In [ ]:
import torch
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset

from data_utils import classification_error, display_confusion_matrix, object_from_json_url
from data_utils import LFWUtils, StandardScaler
from image_utils import make_image

## More Tensors and Why They're Awesome

Multi-dimensional slicing is definitely a nice property of tensors, but what really sets them apart is their ability to keep track of all the operations performed on them using _computational graphs_.

If we define a tensor and set its `requires_grad` parameter to `True` we unlock some really nice properties that we can use for training neural networks.

One of these properties is the ability to automatically calculate derivatives (OMG, calculus!) of functions defined in terms of our tensor.

Let's investigate.

### Easy Calculus and Free Derivatives

Let's pretend we have the following function:

$f(x) = x^4 - 0.7x^3 - 2x^2 + x + 1$

And we want to find out when the function achieves its maximum and minimum values, when it equals $0$, or when it equals $0.5$.

We can plot it, and easily approximate those values visually:

In [ ]:
def peaks(x):
  return x**4 - 0.7*x**3 - 2*x**2 + x + 1

In [ ]:
# linspace is range()'s cousin, but for floats 
#   and where the 3rd argument specifies number of steps, not length of steps

x = torch.linspace(-1.3, 1.6, 300)
y = peaks(x)

plt.plot(x, y)
plt.plot([-1.3, 1.6], [0,0], '-')
plt.plot([-1.3, 1.6], [0.5, 0.5], '-')
plt.show()

Looks like local minimum and maximum values are approximately:
- $x = -0.9$ (global minimum)
- $x = 0.2$ (global maximum)
- $x = 1.2$ (local minimum)

It crosses $y = 0$ at:
- $x = -1.2$
- $x = -0.6$

And, it crosses $y=0.5$ a bunch of times, so we'll look at that later.

We can calculate exact values for these points in our graph if we define $x$ and $y$ as tensors and enable their `auto_grad` functionality.

In [ ]:
xt = torch.linspace(-1.3, 1.6, 8000, requires_grad=True)
yt = peaks(xt)
yt.backward(torch.ones_like(xt))

dydx = xt.grad
print("derivatives:", dydx[:5])

minmax_idx = (dydx.abs() < 9e-4)
minmax_y = yt[minmax_idx]
minmax_x = xt[minmax_idx]

plt.plot(x, y)
plt.plot(minmax_x.tolist(), minmax_y.tolist(), 'o')
plt.show()

print("min/max:", minmax_x, minmax_y)

### Wait. What?

Let's look at the individual commands at the cell above.

`xt`: this is a $1D$ tensor of shape $8000$ with value from $-1.3$ to $1.6$.

`yt`: this is a $1D$ tensor of shape $8000$ which holds the results of calling `peaks()` on every value of `xt`.

`yt.backwards(torch.ones_like(xt))`: this calculates the derivatives (slope) of the equation `peak()` for every point of `yt` and `xt`. The `torch.ones_like(xt)` parameter is a bit unconventional and usually we'll just call `backwards()` without any parameters. It's necessary here because instead of asking for the derivative of an equation at one specific point, we want to get the derivatives for all points in our `xt` range tensor.

`dydx = xt.grad`: after calling `backward()` on a tensor (`yt`) that depends on tensors with `requires_grad` (`xt`), the tensors with `requires_grad` will have their gradients/slope store in the `grad` member variable.

`minmax_idx = (dydx.abs() < 9e-4)`: since our function is being evaluated on a discrete set of values inside `xt`, we might not have the exact `xt` that gives an exact slope of $0$, so `dydx.abs() < 9e-4` is a boolean indexing of all values of dydx that are really close to $0$.

`minmax_y = yt[minmax_idx]` and `minmax_x = xt[minmax_idx]`: this gets the actual `x` and `y` values where the slope of `peaks()` is really really close to $0$.

### Finding Zero

We found `x` and `y` values for when our `peaks()` function is at its `max` and `min` values.

If we want to find when our function is $0$ we can use a little trick and just square it. This will turn any $0$ crossing into a min, and we can repeat the same process as above.

`yt = peaks(xt).pow(2)`: this squares our function, so _y-axis_ crossings become minimum values.

`zeros_idx = ((dydx.abs() < 0.005) & (yt < 1e-7))`: we add an extra condition to the boolean index, so we only plot the minimum values where the derivate is $0$ and `yt` is close to $0$.

In [ ]:
xt = torch.linspace(-1.3, 1.6, 8000, requires_grad=True)
yt = peaks(xt).pow(2)
yt.backward(torch.ones_like(xt))

dydx = xt.grad
print("derivatives:", dydx[:5])

zeros_idx = ((dydx.abs() < 0.005) & (yt < 1e-7))
zeros_x = xt[zeros_idx]
zeros_y = yt[zeros_idx]

plt.plot(x, y)
plt.plot(zeros_x.tolist(), zeros_y.tolist(), 'o')
plt.show()

print("zeros:", zeros_x, zeros_y)

### Finding other values

If we want to find what values of `xt` give a specific value for `yt` we can use a similar trick.

We shift the function up or down to make that `yt` value become $0$, then square the function and repeat the steps as above.

For example, to find values of `xt` that make `peaks()` equal to $0.5$, we subtract $0.5$ and square `peaks()`.

`yt2 = yt.subtract(0.5).pow(2)`: this is the function we use to take the derivative now.

In [ ]:
xt = torch.linspace(-1.3, 1.6, 8000, requires_grad=True)
yt = peaks(xt)
yt2 = yt.subtract(0.5).pow(2)
yt2.backward(torch.ones_like(xt))

dydx = xt.grad
print("derivatives:", dydx[:5])

y05_idx = ((dydx.abs() < 0.005) & (yt2 < 2e-7))
y05_x = xt[y05_idx]
y05_y = yt[y05_idx]

plt.plot(x, y)
plt.plot(y05_x.tolist(), y05_y.tolist(), 'o')
plt.show()

print("y=0.5:", y05_x, y05_y)

### Solving for min/max iteratively

Our `peaks()` function is pretty simple, as it only depends on one variable, `x`, and the range we're calculating it over is pretty small, $[-1.2, 1.6]$.

What if our `peaks()` function was more complex and it took minutes to calculate? How can we find its `min` or `max` values?

This is the more common case for `grad` and `backward()`. We evaluate a function once, at one specific input value, and calculate which direction it should move in order to increase or decrease the value of our function.

We can use the `peaks()` function to illustrate. Let's calculate the value of `x` that gives the smallest value for `peaks(x)`.

`xm`: this is the current guess for the value of `x` which gives the smallest value for `peaks()`. We'll initialize it at $0.15$, which is the halfway point of our `x` range.

`xms` and `yms`: these will hold the progression of the `xm` and `ym` variables as they move towards their objectives.

`ym`: the value of `peaks()` at the current `xm`.

`backwards()`: calculate the slope of `ym` with respect to its inputs.

`xm = xm + 0.1 * xm.grad`: update `xm` according to the slope of `peaks()` at `xm`. If the slope is positive, decrease `xm`, if the slope is negative, increase `xm`. This will move `x_m` towards a minimum value of `peaks()`. If we wanted to move towards a maximum value, we increase `xm` for positive slopes and decrease it for negative slopes.

The $0.1$ factor determines how big our steps should be when we update `xm`. There's a tradeoff here: large steps can get to the desired value quicker, but can also totally skip the desired value and end up in some non-desired part of our equation. Small steps, on the other hand, take a little longer to find the objective, but usually converge on the correct value.

`xm.retain_grad()`: we're again using tensors for educational purposes here, and accumulating gradients in a unconventional way. We have to call this to make sure we can later access the gradient of something that was itself calculated from a gradient.

A tensor's `item()` member function just returns that tensor's value as a regular Python number. Similarly, if we want to get a tensor as a regular Python list we can call its `tolist()` function.

In [ ]:
xs = []
ys = []

xm = torch.tensor(0.15, requires_grad=True)

ym = peaks(xm)
ym.backward()
print(xm.item(), ym.item(), xm.grad)

xs.append(xm.item())
ys.append(ym.item())

xm = xm - 0.1 * xm.grad
xm.retain_grad()

ym = peaks(xm)
ym.backward()
print(xm.item(), ym.item(), xm.grad)

xs.append(xm.item())
ys.append(ym.item())

# TODO: more steps

### X's journey

We saved all of the intermediate values of `xm` and `ym` so we can plot them here:

In [ ]:
plt.plot(x, y)
plt.scatter(xs, ys, marker='o', s=14, c='r')
plt.show()
xs[-1], ys[-1]

### Taking all the steps

We took one step. We could loop and take $10$ steps, or take as many steps as are necessary to get to the closest max/min value of our function.

Let's add a loop to the cell above that repeats the following:

- calculate `ym`
- save `xm` and `ym`
- calculate `gradient`
- update `xm`
- repeat

## Ok, so what ?

Neural Networks is what, because now we have the most important ingredient for training a neural network to perform regression.

We know how to load data into a `DataFrame`, once we pass this data through a neural network with random values for its parameters, we can calculate the `error` of our cost function in relation to all of the parameters of the network, and then calculate which direction to move all of the parameters to decrease our error.

Let's load the housing prices dataset from `HW03`.

As always, we'll encode and scale our data if needed, and then we'll use the `train_test_split()` function to split our `DataFrame` into $2$ separate datasets, a training dataset with $80\%$ of the rows, and a test dataset with $20\%$.

In [ ]:
# Define the location of the json file here
HOUSES_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024F-H/9103-utils/main/datasets/json/LA_housing.json"

houses_info = object_from_json_url(HOUSES_FILE)

houses_raw_df = pd.DataFrame.from_records(houses_info)

house_scaler = StandardScaler()
houses_df = house_scaler.fit_transform(houses_raw_df)

houses_train, houses_test = train_test_split(houses_df, test_size=0.2)

houses_train.head()

### Create features

Just like with the `LinearRegression` models, we have to separate our independent features and our outcome feature.

This time we put them both into tensors.

The `x` tensor holds all of the independent features for all of the data points, and the `y` tensor their corresponding outcomes (prices).

In [ ]:
train_features = houses_train.drop(columns=["value"])
train_values = houses_train["value"]

x_train = torch.Tensor(train_features.values)
y_train = torch.Tensor(train_values.values)

### Define our model

We'll use a very basic neural network model that has an input layer with a neuron for each feature, and a single output neuron for the price prediction.

Something like this:

# ADD IMAGE HERE

Where the initial values for the model parameters are selected at random by default.

We can iterate over out model's parameters and print their shapes, or calculate overall number of parameters using the `numel()` function of each parameter.

In [ ]:
model = nn.Linear(len(train_features.columns), 1)

psum = 0

for p in model.parameters():
  print(p.shape)
  psum += p.numel()

print("number of parameters:", psum)

### Set up training

This will look similar to the iterative approach for finding the minimum value of a function we saw above.

For each step of our iteration we will:

- calculate a price prediction for all of the rows in our dataset
- calculate the overall error for all of the price predictions
- calculate the derivative of this error with respect to the model parameters
- update model parameters to decrease error
- repeat

A few things to note about this process:

1\. We are calculating all of the predictions for all of our rows of data with a single call: `y = model(x)`. `PyTorch` models are smart and they know we want to do the same thing for all of the rows. This optimizes and parallelizes the process.

2\. The cost function (called `loss` here) is the `L2` distance between all price predictions and all actual prices in our dataset calculated in one go. It's a single number we can take the derivative of. We could skip the square root, but this way our units stay consistent and error is in standard deviations.

3\. The parameters we are optimizing and updating at each iteration aren't our features, but the weights and thresholds of each of our $6$ neurons, which have `requires_grad` turned on by default. At each step we update the model's parameters with `p.data.sub_(p.grad.data * learning_rate)`. This is the very bureaucratic form of doing something like: `p -= p * lr`. Since we are dealing with parameter tensors that keep all kinds of extra information about their values, we have to operate on their `data` members.

4\. Once we have used the parameters gradients to update our model we have to clear them by calling `grad.zero_()`. We'll see why soon, but by default if we are reusing the same tensors (in this case our model's parameters) we have to make sure they don't accumulate gradients.

In [ ]:
learning_rate = 1e-2

for c in range(32):
  y_pred = model(x_train)
  loss = (y_pred - y_train).pow(2).mean().pow(0.5)
  loss.backward()
  if c % 4 == 0:
    print(c, loss.item())

  for p in model.parameters():
    p.data.sub_(p.grad.data * learning_rate)
    p.grad.zero_()

### Interpretation

What's happening in the above cell?

What happens if we keep running it over and over?

### Checking the train dataset

Once we're happy with the training, we can get predictions for all of our houses in dollars by running the model and reversing the scaling:

In [ ]:
y_std = pd.DataFrame(model(x_train).tolist(), columns=["value"])
y_usd = house_scaler.inverse_transform(y_std)

y_usd.head()

### Growing the Network

The error we were getting above was around $1.0$ standard deviation. That's not bad, but it's also not good.

If we want to improve our model we can try adding layers to our Neural Network. We just have to make sure we add an activation function between the neurons. These are the functions that keep our model parameters within a nice, expected, range.

This is how we build the following network:

# ADD IMAGE HERE

In [ ]:
model =  nn.Sequential(
  nn.Linear(len(train_features.columns), len(train_features.columns)),
  nn.Sigmoid(),
  nn.Linear(len(train_features.columns), 1),
)

# TODO: calculate the number of parameters

### So... many... parameters

How many parameters do we have now? We might not want to keep updating them ourselves.

Relying on a for loop to get all the parameters and remembering to call `grad.zero_()` at the right time is just prone to errors and inefficiencies.

Luckily, `PyTorch` has some optimizers we can use. They usually take our model as an input, along with some other parameters, and give us a simpler interface to control the optimization process.

### Initialize Optimizer

We're going to use one of the simpler optimizers to performs [_stochastic gradient descent_](https://en.wikipedia.org/wiki/Stochastic_gradient_descent). Gradient descent is the official name of the algorithm that calculates which way to update our parameters given the slope of our cost function. _Stochastic_ means that it will use statistics to make certain decisions that don't have exact solutions.

The documentation for the [`SGD` optimizer](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html) has more info about the algorithm and the parameters it takes.

Other than simplifying our training code, these pre-built optimizers also perform dynamic learning rate adjustment and some other tricks that make our overall process not so sensitive to an exact learning rate.

The `PyTorch` library also has a number of [other optimizers](https://pytorch.org/docs/stable/optim.html#algorithms) useful for performing gradient descent. In addition to `SGD()` we can also try [Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html) or [Adagrad](https://pytorch.org/docs/stable/generated/torch.optim.Adagrad.html).

In [ ]:
learning_rate = 1e-2
optim = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Train it

We can train our new model, just like before, except now the training loop should be a little bit simpler.

We still have to call `zero_grad()`, but only on the optimizer. It will take care of clearing the gradients for each of the model's parameters for us.

And, after we calculate the slope of our cost function, we call `optim.step()`, so the optimizer can update the parameters with a new slope value.

In [ ]:
for c in range(32):
  optim.zero_grad()
  y_pred = model(x_train)
  loss = (y_pred - y_train).pow(2).mean().pow(0.5)
  loss.backward()
  optim.step()
  if c % 4 == 0:
    print(c, loss.item())

### Test dataset

We can still adjust a lot of parameters here, but before we spend too much time on this model, let's run it on the test dataset and calculate the average loss on data that wasn't used for training to see if the model is over-fitting.

We'll load the test data, run the mode and calculate loss.

In [ ]:
test_features = houses_test.drop(columns=["value"])
test_values = houses_test["value"]

x_test = torch.Tensor(test_features.values)
y_test = torch.Tensor(test_values.values)

### `no_grad()`

We can tell `PyTorch` to momentarily stop calculating slopes/gradients when we are not training and just want to use the model to predict prices by using the `torch.no_grad()` function to create a block of code where model runs faster and more carefree.

In [ ]:
with torch.no_grad():
  y_pred = model(x_test)
  loss = (y_pred - y_test).pow(2).mean().pow(0.5)
  print(loss.item())

### Interpretation

This isn't bad.

The absolute value of the error is kind of large, but the test dataset error is comparable to the training dataset error, which is a good indication that the model is not over-fitting.

### Hyperparameters

We can spend some time adjusting the model, adding layers, changing the optimizer, the learning rate, experimenting with the optimizer's parameters, etc.

This process is usually referred to as hyperparameter tuning, since we're picking parameters that will help us calculate the parameters of our neural network.

Here's a cell with all of the steps combined. We can play with the network architecture and parameters a little.

In [ ]:
## Define Model
model =  nn.Sequential(
  nn.Linear(len(train_features.columns), 10 * len(train_features.columns)),
  nn.Sigmoid(),
  # TODO: add layers

  nn.Linear(10 * len(train_features.columns), 1),
)
# TODO: calculate the number of parameters

## Define Optimizer
learning_rate = 1e-1
# TODO: adjust parameters, add parameters, change optimizer
optim = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Load Data
x_train = torch.Tensor(train_features.values)
y_train = torch.Tensor(train_values.values)
x_test = torch.Tensor(test_features.values)
y_test = torch.Tensor(test_values.values)

## Train Model
for c in range(32):
  optim.zero_grad()
  y_pred = model(x_train)
  loss = (y_pred - y_train).pow(2).mean().pow(0.5)
  loss.backward()
  optim.step()
  if c % 4 == 0:
    print(c, loss.item())

## Evaluate Model
with torch.no_grad():
  y_pred = model(x_train)
  loss_train = (y_pred - y_train).pow(2).mean().pow(0.5)

  y_pred = model(x_test)
  loss_test = (y_pred - y_test).pow(2).mean().pow(0.5)

  print("\ntrain loss:", loss_train.item(), "\ntest loss:", loss_test.item())

### Interpretation

Our model is definitely learning. It doesn't perform very well, but that's not entirely surprising.

Usually what makes the biggest difference in these kinds of models is the size of the training dataset, and our houses dataset only has $4000$ samples.

## Images

Load the Labeled Faces in the Wild dataset

In [ ]:
# 0. Split data into train/test and do any pre-processing

# 1. Dataloader? Can computer handle entire dataset at once?

# 2. What's my architecture/model?

# 3. What's my cost/loss function?

# 4. What's my optimizer?

# 5. What's my evaluation function?

In [ ]:
train, test = LFWUtils.train_test_split(0.333)

x_train = torch.Tensor(train["pixels"])
y_train = torch.Tensor(train["labels"]).long()

x_test = torch.Tensor(test["pixels"])
y_test = torch.Tensor(test["labels"]).long()

In [ ]:
for idx in range(0, len(train["pixels"]), 100):
  display(make_image(train["pixels"][idx], 130))
  print(train["labels"][idx], LFWUtils.LABELS[train["labels"][idx]])

## DataLoader

- batch
- random

In [ ]:
class FaceDataset(Dataset):
  def __init__(self, imgs, labels):
    self.imgs = imgs
    self.labels = labels

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return self.imgs[idx], self.labels[idx]

In [ ]:
train_dataloader = DataLoader(FaceDataset(x_train, y_train), batch_size=256, shuffle=True)
test_dataloader = DataLoader(FaceDataset(x_test, y_test), batch_size=128)

This is useful for confusion matrix and calculating accuracy/classification error

In [ ]:
def get_labels(model, data):
  model.eval()
  with torch.no_grad():
    data_labels = []
    pred_labels = []
    for x, y in data:
      y_pred = model(x).argmax(dim=1)
      data_labels += [l.item() for l in y]
      pred_labels += [l.item() for l in y_pred]
    return data_labels, pred_labels

In [ ]:
model = nn.Linear(x_train.shape[1], len(y_train.unique()))

learning_rate = 1e-5
optim = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
for e in range(32):
  for x, y in train_dataloader:
    optim.zero_grad()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    loss.backward()
    optim.step()

  if e % 4 == 0:
    train_labels, train_predictions = get_labels(model, train_dataloader)
    test_labels, test_predictions = get_labels(model, test_dataloader)
    train_error = classification_error(train_labels, train_predictions)
    test_error = classification_error(test_labels, test_predictions)
    print(f"Epoch: {e} loss: {loss.item():.4f}, train error: {train_error:.4f}, test error: {test_error:.4f}")

In [ ]:
train_labels, train_predictions = get_labels(model, train_dataloader)
test_labels, test_predictions = get_labels(model, test_dataloader)

display_confusion_matrix(train_labels, train_predictions, display_labels=LFWUtils.LABELS)
display_confusion_matrix(test_labels, test_predictions, display_labels=LFWUtils.LABELS)

In [ ]:
model =  nn.Sequential(
  nn.Dropout(0.2),
  nn.Linear(x_train.shape[1], x_train.shape[1] // 8),
  nn.ReLU(),

  nn.Dropout(0.2),
  nn.Linear(x_train.shape[1] // 8, len(y_train.unique())),
)

learning_rate = 1e-5
optim = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
for e in range(32):
  model.train()
  for x, y in train_dataloader:
    optim.zero_grad()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    loss.backward()
    optim.step()

  if e % 4 == 0:
    train_labels, train_predictions = get_labels(model, train_dataloader)
    test_labels, test_predictions = get_labels(model, test_dataloader)
    train_error = classification_error(train_labels, train_predictions)
    test_error = classification_error(test_labels, test_predictions)
    print(f"Epoch: {e} loss: {loss.item():.4f}, train error: {train_error:.4f}, test error: {test_error:.4f}")

### Interpretation

train and test eval function diverged. but both keep decreasing. might be ok.

In [ ]:
train_labels, train_predictions = get_labels(model, train_dataloader)
test_labels, test_predictions = get_labels(model, test_dataloader)

print("train error", f"{classification_error(train_labels, train_predictions):.4f}")
print("test error", f"{classification_error(test_labels, test_predictions):.4f}")

display_confusion_matrix(train_labels, train_predictions, display_labels=LFWUtils.LABELS)
display_confusion_matrix(test_labels, test_predictions, display_labels=LFWUtils.LABELS)

print("test: top precision", LFWUtils.top_precision(test_labels, test_predictions))
print("test: top recall", LFWUtils.top_recall(test_labels, test_predictions))